# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations (lats go 90 to -90 and lngs go 180 to -180)
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

In [3]:
# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    #If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city) 

# Print the city count to confirm sufficient count
len(cities)

635

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
lat = []
lng = []
temp = []
max_temp = []
humidity = []
clouds = []
wind = []
country = []
date = []

url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [5]:
for city in cities:
    
    query = query_url + city
    
    #Try to grab the city
    try:
        response = requests.get(query).json()         
        print(f'Processing Record | {city}')    
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        temp.append(response['main']['temp'])
        max_temp.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        clouds.append(response['clouds']['all'])
        wind.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        
    
    except:
        print(f'City not found. Skipping...')
        pass
    
    time.sleep(1)

Processing Record | rikitea
Processing Record | ponta delgada
Processing Record | nanortalik
Processing Record | fortuna
Processing Record | pacifica
Processing Record | grayvoron
Processing Record | thompson
Processing Record | tianpeng
Processing Record | lavrentiya
Processing Record | ushuaia
Processing Record | cedar city
Processing Record | werda
Processing Record | nizhneyansk
City not found. Skipping...
Processing Record | vila velha
Processing Record | tura
Processing Record | arraial do cabo
Processing Record | avarua
Processing Record | saint-philippe
Processing Record | ribeira grande
Processing Record | quiindy
Processing Record | bengkulu
Processing Record | isangel
Processing Record | louisbourg
City not found. Skipping...
Processing Record | chokurdakh
Processing Record | busselton
Processing Record | bentiu
Processing Record | vaini
Processing Record | eyl
Processing Record | butaritari
Processing Record | codrington
Processing Record | abonnema
Processing Record | inyo

Processing Record | imeni babushkina
Processing Record | mount isa
Processing Record | iranshahr
Processing Record | tsihombe
City not found. Skipping...
Processing Record | attawapiskat
City not found. Skipping...
Processing Record | severo-yeniseyskiy
Processing Record | hofn
Processing Record | matara
Processing Record | mareeba
Processing Record | sistranda
Processing Record | reconquista
Processing Record | ancud
Processing Record | dom pedro
Processing Record | yarmouth
Processing Record | dagda
Processing Record | opuwo
Processing Record | los llanos de aridane
Processing Record | puerto escondido
Processing Record | pisco
Processing Record | bathsheba
Processing Record | kudahuvadhoo
Processing Record | mendahara
City not found. Skipping...
Processing Record | sitka
Processing Record | alappuzha
Processing Record | whitianga
Processing Record | teya
Processing Record | faanui
Processing Record | gravdal
Processing Record | kamenskoye
City not found. Skipping...
Processing Recor

Processing Record | pozo colorado
Processing Record | revelstoke
Processing Record | tuatapere
Processing Record | scarborough
Processing Record | odugathur
City not found. Skipping...
Processing Record | mehamn
Processing Record | mugur-aksy
Processing Record | kirakira
Processing Record | lagos
Processing Record | sivaki
Processing Record | northam
Processing Record | stamsund
City not found. Skipping...
Processing Record | sinaloa
Processing Record | port hedland
Processing Record | san cristobal
Processing Record | ust-tsilma
Processing Record | necochea
Processing Record | onega
Processing Record | miyako
Processing Record | bad freienwalde
Processing Record | westport
Processing Record | lakes entrance
Processing Record | masjed-e soleyman
City not found. Skipping...
Processing Record | brumado
Processing Record | marzuq
Processing Record | asfi
Processing Record | nguiu
City not found. Skipping...
Processing Record | pacific grove
Processing Record | arlit
Processing Record | li

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [33]:
# create a data frame
weather_dict = {
    "City": cities,
    "Lat": lat,
    "Lng": lng,
    "Temperature": temp,
    "Max Temp": max_temp,
    "Humidity": humidity,
    "Cloudiness": clouds,
    "Wind Speed": wind,
    "Country": country,
    "Date": date
}

weather_data = pd.DataFrame.from_dict(weather_dict)
#weather_data = pd.DataFrame.from_dict(weather_dict, orient="index")

ValueError: arrays must all be same length

In [30]:
weather_data["Date"] = pd.to_datetime(weather_data["Date"],unit="s")

KeyError: 'Date'

In [29]:
weather_data

KeyError: 'Date'

In [ ]:
#save to a csv
#weather_data.to_csv("Output/cities.csv", index=False, header=True)

In [12]:
# Using the aggregation method, produce a data summary statistics 
summary_stats = weather_data.agg(["count","mean","std","min","max"])
summary_stats

,0,1,2,3,4,5,6,7,8,9,...,625,626,627,628,629,630,631,632,633,634
count,10,10,10,10,10,10,10,10,10,10,...,1,1,1,1,1,1,1,1,1,1


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#df.loc >= 100

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(df['Lat'], df['Temperature'])
plt.ylabel("Temperature (F)")
plt.xlabel("Latitude")
plt.title("City Lattitude vs. Temperature (Date)")
plt.show()

#this is a markdown cell to explain what the code is analyzing 

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(df['Lat'], df['Humidity'])
plt.ylabel("Humidity")
plt.xlabel("Latitude")
plt.title("City Lattitude vs. Humidity (Date)")
plt.show()

#this is a markdown cell to explain what the code is analyzing 

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(df['Lat'], df['Cloudiness'])
plt.ylabel("Cloudiness")
plt.xlabel("Latitude")
plt.title("City Lattitude vs. Cloudiness (Date)")
plt.show()

#this is a markdown cell to explain what the code is analyzing 

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(df['Lat'], df['Wind Speed'])
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.title("City Lattitude vs. Wind Speed (Date)")
plt.show()

#this is a markdown cell to explain what the code is analyzing 

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = df["column"]
y_values = df["column"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(x coord, y coord),fontsize=15,color="red")
plt.xlabel('Label')
plt.ylabel('Label')
print(f'The r-squared is: {rvalue**2}')
plt.show()
fig.savefig('northern_hemisphere_max_temp_vs_latitude_linereg.png')

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = df["column"]
y_values = df["column"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(x coord, y coord),fontsize=15,color="red")
plt.xlabel('Label')
plt.ylabel('Label')
print(f'The r-squared is: {rvalue**2}')
plt.show()
fig.savefig('southern_hemisphere_max_temp_vs_latitude_linereg.png')

#this is a markdown cell to explain what the linear regression is modeling and any other analysis (results of the regression) 

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = df["column"]
y_values = df["column"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(x coord, y coord),fontsize=15,color="red")
plt.xlabel('Label')
plt.ylabel('Label')
print(f'The r-squared is: {rvalue**2}')
plt.show()
fig.savefig('northern_hemisphere_humid_vs_latitude_linereg.png')

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = df["column"]
y_values = df["column"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(x coord, y coord),fontsize=15,color="red")
plt.xlabel('Label')
plt.ylabel('Label')
print(f'The r-squared is: {rvalue**2}')
plt.show()
fig.savefig('southern_hemisphere_humid_vs_latitude_linereg.png')

#this is a markdown cell to explain what the linear regression is modeling and any other analysis (results of the regression) 

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = df["column"]
y_values = df["column"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(x coord, y coord),fontsize=15,color="red")
plt.xlabel('Label')
plt.ylabel('Label')
print(f'The r-squared is: {rvalue**2}')
plt.show()
fig.savefig('northern_hemisphere_cloud_vs_latitude_linereg.png')

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = df["column"]
y_values = df["column"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(x coord, y coord),fontsize=15,color="red")
plt.xlabel('Label')
plt.ylabel('Label')
print(f'The r-squared is: {rvalue**2}')
plt.show()
fig.savefig('southern_hemisphere_cloud_vs_latitude_linereg.png')

#this is a markdown cell to explain what the linear regression is modeling and any other analysis (results of the regression) 

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = df["column"]
y_values = df["column"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(x coord, y coord),fontsize=15,color="red")
plt.xlabel('Label')
plt.ylabel('Label')
print(f'The r-squared is: {rvalue**2}')
plt.show()
fig.savefig('northern_hemisphere_wind_vs_latitude_linereg.png')

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = df["column"]
y_values = df["column"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(x coord, y coord),fontsize=15,color="red")
plt.xlabel('Label')
plt.ylabel('Label')
print(f'The r-squared is: {rvalue**2}')
plt.show()
fig.savefig('southern_hemisphere_wind_vs_latitude_linereg.png')

#this is a markdown cell to explain what the linear regression is modeling and any other analysis (results of the regression) 